In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from os import urandom

In [2]:
def xor(*args, **kwargs):
    strs = [s for s in args]

    length = kwargs.pop("length", "max")
    if isinstance(length, int):
        length = length
    elif length == "max":
        length = max(len(s) for s in strs)
    elif length == "min":
        length = min(len(s) for s in strs)
    else:
        raise ValueError("Invalid value for length parameter")

    def xor_indices(index):
        b = 0
        for s in strs:
            b ^= s[index % len(s)]
        return b

    return bytes([xor_indices(i) for i in range(length)])

In [3]:
class someService:
    def __init__(self):
        self.key = urandom(16)

    def encrypt_user_data(self, data: str) -> bytes:
        final_data = self._parse_user_data(data)

        return self._encrypt(pad(final_data, AES.block_size))

    def is_admin(self, encrypted_data: bytes) -> bool:
        iv = encrypted_data[:16]
        ct = encrypted_data[16:]
        user_data = self._decrypt(ct, iv)

        return b";admin=true;" in user_data

    def _parse_user_data(self, data: str) -> bytes:
        data = data.replace(";", "\\;").replace("=", "\\=")
        data = (
            "comment1=cooking%20MCs;userdata="
            + data
            + ";comment2=%20like%20a%20pound%20of%20bacon"
        )

        return data.encode()

    def _encrypt(self, data: bytes) -> bytes:
        cipher = AES.new(self.key, AES.MODE_CBC)

        return cipher.encrypt(data)

    def _decrypt(self, data: bytes, iv: bytes) -> bytes:
        cipher = AES.new(self.key, AES.MODE_CBC, iv)

        return unpad(cipher.decrypt(data), AES.block_size)

In [4]:
def bit_flip(
    target_decryption: bytes, current_decryption: bytes, prev_ct: bytes
) -> bytes:
    keystream = xor(current_decryption, prev_ct)
    flipped_block = xor(target_decryption, keystream)
    new_ct = prev_ct[:16] + flipped_block + prev_ct[32:]

    return new_ct

In [5]:
service = someService()
dummy = "blaahXadminXtrue"
target = "blaah;admin=true"

encrypted_dummy = service.encrypt_user_data(dummy)

flipped_block = bit_flip(target.encode(), dummy.encode(), encrypted_dummy[16:32])
flipped_encryption = encrypted_dummy[:16] + flipped_block + encrypted_dummy[32:]

admin = service.is_admin(flipped_encryption)
print(f"Admin? {admin}")

Admin? True
